In [16]:
!pip install torch==1.6.0
!pip install opencv-python
!pip install torchvision==0.7.0
!pip install albumentations
!pip install tensorflow
!pip install pytorch-lightning

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 5.9 MB 10.1 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.2.2
    Uninstalling torchvision-0.2.2:
      Successfully uninstalled torchvision-0.2.2
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.5 MB 1.2 MB/s 
     |████████████████████████████████| 2.9 MB 2.1 MB/s 
     |████████████████████████████████| 723 kB 4.7 MB/s 
     |████████████████████████████████| 883 kB 4.1 MB/s 
     |████████████████████████████████| 82 kB 1.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893258 sha256=012e87548fcdc737b5caae9fd67fb1363301bb419418474a742bf1e0c605ef0f
  Stored in directory: /home/user/.cache/pip/wheels/49/25/98/cdea9c79b2d9a22ccc59540b1784b67f06b633378e97f58da2
Successfully built sacremoses
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
!pip install torchlibrosa

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import typing as tp
import yaml
import random
import os
import sys
import soundfile as sf
import librosa
import cv2
import matplotlib.pyplot as plt
import time
import glob

import pickle
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# import resnest.torch as resnest_torch

from torchvision import models
from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation
# from mixup import do_mixup
import random
from timm.models.efficientnet import tf_efficientnet_b0_ns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
# from resnet import ResNet, Bottleneck

from albumentations.core.transforms_interface import DualTransform, BasicTransform
import albumentations as A
from audiomentations import Compose, AddGaussianSNR, AddGaussianNoise, PitchShift, AddBackgroundNoise, AddShortNoises, Gain, PitchShift
from functools import partial


from sklearn.model_selection import StratifiedKFold

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

## util

In [2]:
SEED=1213
PERIOD = 10
OK_RANGE = 100
SPECIES_NUM = 24
EPOCH = 25
OUTPUT_DIR = './output/'
HOP_LEN = 512
SR = 48000
EPSILON_FP16 = 1e-5
MEL_BIN = 384
ASOBI = 0
NO_LABEL = -1

config_set = {
    'dataset': {
          'name': 'SpectrogramDataset',
          'params': {
            'img_size': 224, 
            'melspectrogram_parameters': {
                'n_fft': 2048,
                'hop_length': 512,
                'win_length': 2048,
                'n_mels': MEL_BIN, 
                'fmin': 50, 
                'fmax': 24000, 
            }
      }
    },
    'loader': {
      'train': {
        'batch_size': 6,
        'shuffle': True,
        'num_workers': 2,
        'pin_memory': True,
        'drop_last': True,
      },
      'valid': {
        'batch_size': 2,
        'shuffle': False,
        'num_workers': 2,
        'pin_memory': True,
        'drop_last': True,
      }
    }
}


In [3]:
config = config_set

In [4]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
set_seed(SEED)

In [5]:
INPUT_ROOT = Path("/home/knikaido/work/Rainforest-Connection/data")
RAW_DATA = INPUT_ROOT / "rfcx-species-audio-detection"
TRAIN_AUDIO_DIR = RAW_DATA / "train"
# TRAIN_RESAMPLED_AUDIO_DIRS = [
#   INPUT_ROOT / "birdsong-resampled-train-audio-{:0>2}".format(i)  for i in range(5)
# ]
TEST_AUDIO_DIR = RAW_DATA / "test"
pinknoise_dir = INPUT_ROOT / 'pinknoise_resample'

In [6]:
def get_augmentation():
    list_of_aug = [
    #         AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.3),
        AddGaussianNoise(p=0.2),
        AddGaussianSNR(p=0.2),
        Gain(min_gain_in_db=-15,max_gain_in_db=15,p=0.3),
        PitchShift(min_semitones=-1, max_semitones=1, p=0.2)
#         AddBackgroundNoise(str(pinknoise_dir),p=0.2),
#         AddShortNoises(str(pinknoise_dir),min_time_between_sounds=0.0, max_time_between_sounds=15.0, burst_probability=0.5, p=0.6)
    ]
    return Compose(list_of_aug)

In [7]:
train_df = pd.read_pickle(RAW_DATA / "train_gby_wav_raw_denoise.pkl")
train_df

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,name
0,003bec244,14,1,44.5440,2531.250,45.1307,5531.25,/home/knikaido/work/Rainforest-Connection/data...
1,006ab765f,23,1,39.9615,7235.160,46.0452,11283.40,/home/knikaido/work/Rainforest-Connection/data...
2,007f87ba2,12,1,39.1360,562.500,42.2720,3281.25,/home/knikaido/work/Rainforest-Connection/data...
3,0099c367b,17,4,51.4206,1464.260,55.1996,4565.04,/home/knikaido/work/Rainforest-Connection/data...
4,009b760e6,10,1,50.0854,947.461,52.5293,10852.70,/home/knikaido/work/Rainforest-Connection/data...
...,...,...,...,...,...,...,...,...
1211,fe8d9ac40,13,1,53.4720,93.750,54.0960,843.75,/home/knikaido/work/Rainforest-Connection/data...
1212,fea6b438a,4,1,43.5787,2531.250,45.7653,4031.25,/home/knikaido/work/Rainforest-Connection/data...
1213,ff2eb9ce5,0,1,15.2267,5906.250,16.0213,8250.00,/home/knikaido/work/Rainforest-Connection/data...
1214,ffb8d8391,5,1,14.3467,4781.250,16.6987,10406.20,/home/knikaido/work/Rainforest-Connection/data...


In [8]:
train_gby = pd.read_pickle(RAW_DATA / "train_gby_denoise.pkl")
train_gby

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,name
0,003bec244,[14],[1],[44.544],[2531.25],[45.1307],[5531.25],/home/knikaido/work/Rainforest-Connection/data...
1,006ab765f,[23],[1],[39.9615],[7235.16],[46.0452],[11283.4],/home/knikaido/work/Rainforest-Connection/data...
2,007f87ba2,[12],[1],[39.135999999999996],[562.5],[42.272],[3281.25],/home/knikaido/work/Rainforest-Connection/data...
3,0099c367b,[17],[4],[51.4206],[1464.26],[55.1996],[4565.04],/home/knikaido/work/Rainforest-Connection/data...
4,009b760e6,[10],[1],[50.0854],[947.461],[52.5293],[10852.7],/home/knikaido/work/Rainforest-Connection/data...
...,...,...,...,...,...,...,...,...
1127,fe8d9ac40,[13],[1],[53.472],[93.75],[54.096000000000004],[843.75],/home/knikaido/work/Rainforest-Connection/data...
1128,fea6b438a,[4],[1],[43.5787],[2531.25],[45.7653],[4031.25],/home/knikaido/work/Rainforest-Connection/data...
1129,ff2eb9ce5,[0],[1],[15.2267],[5906.25],[16.0213],[8250.0],/home/knikaido/work/Rainforest-Connection/data...
1130,ffb8d8391,[5],[1],[14.3467],[4781.25],[16.6987],[10406.2],/home/knikaido/work/Rainforest-Connection/data...


In [9]:
y, sr = sf.read(train_df['name'][0])
len(y)

2880000

In [10]:
random.sample(list(train_df['name']), 1)[0]

'/home/knikaido/work/Rainforest-Connection/data/rfcx-species-audio-detection/train_denoise/dabd22b08_denoise.flac'

In [11]:
class SpectrogramTrainDataset(data.Dataset):
    def __init__(
        self,
        gby_df: pd.DataFrame,
        setting: tp.Dict
    ):
        self.img_size = setting['img_size']
        self.melspectrogram_parameters = setting['melspectrogram_parameters']
        self.transform = get_augmentation()
        
        self.gby_df = gby_df

    def __len__(self):
        return len(self.gby_df) * 2

    def __getitem__(self, idx: int):
        
        if idx >= len(self.gby_df['name']):
            audio, label = self.get_unlabeled_item(idx)
            # For unlabeled samples, we zero out the classification loss.
            classif_weights = np.zeros(SPECIES_NUM, dtype='f')
        else:
            audio, label= self.get_labeled_item(idx)
            classif_weights = np.ones(SPECIES_NUM, dtype='f')
            
        audio_teacher = np.copy(audio)

        # The 2 samples fed to the 2 models have should have different augmentations.
        audio = self.transform(samples=audio, sample_rate=sr)
        audio_teacher = self.transform(samples=audio_teacher, sample_rate=sr)
        # assert (audio != audio_teacher).any()
        
        return {
            "waveform": audio,
            "teacher_waveform": audio_teacher,
            "target": label,
            "classification_weights": classif_weights
        }
            
    def get_labeled_item(self, idx):

        wav_path = self.gby_df['name'][idx]
        train_element = self.gby_df.iloc[idx]

        y, sr = sf.read(wav_path)

        len_y = len(y)
        effective_length = int(sr * PERIOD)

        tmin = max(SR * (train_element['t_min'] - ASOBI), SR * 0.001)
        tmax = min(SR * (train_element['t_max'] + ASOBI), len_y - ( SR * 0.001 ))      

        start_min = max(0, tmax-effective_length)
        start_max = min(tmin, len_y - effective_length)
        start = np.random.randint(start_min, start_max)
        end = start + effective_length

        y = y[start:end].astype(np.float32)
        y = self.transform(samples=y, sample_rate=sr)

        label = np.zeros(SPECIES_NUM, dtype="f")
        label[train_element['species_id']] = 1

        return y, label

    def get_unlabeled_item(self, idx, random_sample=False):
        real_idx = idx - len(self.gby_df['name'])
        # We want our validation set to be fixed.
#             if self.val:
#                 rec_id = self.recording_ids[real_idx]
#             else:
        wav_path = random.sample(list(self.gby_df['name']), 1)[0]
        y, sr = sf.read(wav_path)

        len_y = len(y)
        effective_length = int(sr * PERIOD)
        max_end = len_y - effective_length
#             if self.val:
#                 # Fixed start for validation. Probaably a better way to do this.
#                 start = int(idx * 16963 % max_end)
#             else:
        start = np.random.randint(0, max_end)
        y = y[start:(start+effective_length)]
        y = y.astype(np.float32)

        label = np.ones(SPECIES_NUM, dtype='f') * NO_LABEL

        return y, label
    
class SpectrogramValidDataset(data.Dataset):
    def __init__(
        self,
        gby_df: pd.DataFrame,
        setting: tp.Dict
    ):
        self.img_size = setting['img_size']
        self.melspectrogram_parameters = setting['melspectrogram_parameters']
        
        self.gby_df = gby_df

    def __len__(self):
        return len(self.gby_df)

    def __getitem__(self, idx: int):
        
        wav_path = self.gby_df['name'][idx]
        train_element = self.gby_df.iloc[idx]
        
        y, sr = sf.read(wav_path)
        
        len_y = len(y)
        effective_length = int(sr * PERIOD)

        tmin = int(SR * train_element['t_min'])
        tmax = int(SR * train_element['t_max'])
        
        #時間かかる
        while(1):
            start = np.random.randint(len_y - effective_length)
            end = start + effective_length
            tgt_len = int((tmax - tmin) * 100 / 100)
            if( (start < tmin and tmin + tgt_len < end) or (start < tmax - tgt_len and tmax < end) ):
                break
        
        y = y[start:end].astype(np.float32)
        
        label = np.zeros(SPECIES_NUM, dtype="f")
        label[train_element['species_id']] = 1

        return y, label

In [12]:
def bce_criterion(y_pred, y_true, isTrain=True):
    y_pred = torch.clamp(y_pred, min=EPSILON_FP16, max=1.0-EPSILON_FP16)
#     loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
    bce_loss = nn.BCELoss()(y_pred, y_true)
    
    if isTrain == True:
        gamma = 2
        alpha = 0.5
        bce_loss = y_true * alpha * (1. - y_pred)**gamma * bce_loss + (1. -  y_true) *  y_pred**gamma * bce_loss
        bce_loss = bce_loss.mean()
    
    return bce_loss

def sigmoid_rampup(current, rampup_length):
    """Exponential rampup from https://arxiv.org/abs/1610.02242"""
    if rampup_length == 0:
        return 1.0
    else:
        current = np.clip(current, 0.0, rampup_length)
        phase = 1.0 - current / rampup_length
        return float(np.exp(-5.0 * phase * phase))

def sigmoid_mse_loss(input_logits, target_logits):
    assert input_logits.size() == target_logits.size()
    input_softmax = torch.sigmoid(input_logits)
#     input_softmax = input_logits
    target_softmax = torch.sigmoid(target_logits)
    num_classes = input_logits.size()[1]
    return F.mse_loss(input_softmax, target_softmax, size_average=False
                     ) / num_classes

class MeanTeacherLoss(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.positive_weight = torch.ones(
            SPECIES_NUM).to('cuda') * 2.0
        self.class_criterion = nn.BCEWithLogitsLoss(
            reduction='none', pos_weight=self.positive_weight)
#         self.class_criterion = bce_criterion
        self.consistency_criterion = sigmoid_mse_loss
        self.consistency_weight = 100
        self.consistency_rampup = 1000

    def make_safe(self, pred):
        pred = torch.where(torch.isnan(pred), torch.zeros_like(pred), pred)
        return torch.where(torch.isinf(pred), torch.zeros_like(pred), pred)
        
    def get_consistency_weight(self, epoch):
        # Consistency ramp-up from https://arxiv.org/abs/1610.02242
        return self.consistency_weight * sigmoid_rampup(
            epoch, self.consistency_rampup)
    
    def forward(self, student_pred, teacher_pred, target, classif_weights, epoch):
        student_pred = self.make_safe(student_pred)
        teacher_pred = self.make_safe(teacher_pred).detach().data

        batch_size = len(target)
        labeled_batch_size = target.ne(NO_LABEL).all(axis=1).sum().item() + 1e-3

        student_classif, student_consistency = student_pred, student_pred
        student_class_loss = (self.class_criterion(
            student_classif, target) * classif_weights / labeled_batch_size).sum()

        consistency_weights = self.get_consistency_weight(epoch)
        consistency_loss = consistency_weights * self.consistency_criterion(
            student_consistency, teacher_pred) / batch_size
        loss = student_class_loss + consistency_loss
        return loss, student_class_loss, consistency_loss, consistency_weights

In [13]:
early_stop_callback = EarlyStopping(
   monitor='valid_epoch_lwlap',
   min_delta=0.00,
   patience=5,
   verbose=True,
   mode='max'
)

In [14]:
def _one_sample_positive_class_precisions(scores, truth):
    num_classes = scores.shape[0]
    pos_class_indices = np.flatnonzero(truth > 0)

    if not len(pos_class_indices):
        return pos_class_indices, np.zeros(0)

    retrieved_classes = np.argsort(scores)[::-1]

    class_rankings = np.zeros(num_classes, dtype=np.int)
    class_rankings[retrieved_classes] = range(num_classes)

    retrieved_class_true = np.zeros(num_classes, dtype=np.bool)
    retrieved_class_true[class_rankings[pos_class_indices]] = True

    retrieved_cumulative_hits = np.cumsum(retrieved_class_true)

    precision_at_hits = (
            retrieved_cumulative_hits[class_rankings[pos_class_indices]] /
            (1 + class_rankings[pos_class_indices].astype(np.float)))
    return pos_class_indices, precision_at_hits

def lwlrap(truth, scores):
    assert truth.shape == scores.shape
    num_samples, num_classes = scores.shape
    precisions_for_samples_by_classes = np.zeros((num_samples, num_classes))
    for sample_num in range(num_samples):
        pos_class_indices, precision_at_hits = _one_sample_positive_class_precisions(scores[sample_num, :], truth[sample_num, :])
        precisions_for_samples_by_classes[sample_num, pos_class_indices] = precision_at_hits

    labels_per_class = np.sum(truth > 0, axis=0)
    weight_per_class = labels_per_class / float(np.sum(labels_per_class))

    per_class_lwlrap = (np.sum(precisions_for_samples_by_classes, axis=0) /
                        np.maximum(1, labels_per_class))
    return per_class_lwlrap, weight_per_class

def lwlap_wrapper(y_true, y_score):
    y_true = y_true.cpu()
    y_score = y_score.cpu()
    y_true = y_true.to('cpu').detach().numpy().copy()
    y_score = y_score.to('cpu').detach().numpy().copy()
    score_class, weight = lwlrap(y_true, y_score)
    score_class = torch.from_numpy(score_class.astype(np.float32)).clone()
    weight = torch.from_numpy(weight.astype(np.float32)).clone()
    return score_class, weight

y_true = np.array([[1, 0, 0], [0, 0, 1]])
y_score = np.array([[0.75, 0.5, 1], [1, 0.2, 0.1]])
y_true = torch.from_numpy(y_true.astype(np.float32)).clone()
y_score = torch.from_numpy(y_score.astype(np.float32)).clone()

score_class, weight = lwlap_wrapper(y_true, y_score)
score = (score_class * weight).sum()
score

tensor(0.4167)

In [15]:

def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.

    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled

def _one_sample_positive_class_precisions(scores, truth):
    num_classes = scores.shape[0]
    pos_class_indices = np.flatnonzero(truth > 0)

    if not len(pos_class_indices):
        return pos_class_indices, np.zeros(0)

    retrieved_classes = np.argsort(scores)[::-1]

    class_rankings = np.zeros(num_classes, dtype=np.int)
    class_rankings[retrieved_classes] = range(num_classes)

    retrieved_class_true = np.zeros(num_classes, dtype=np.bool)
    retrieved_class_true[class_rankings[pos_class_indices]] = True

    retrieved_cumulative_hits = np.cumsum(retrieved_class_true)

    precision_at_hits = (
            retrieved_cumulative_hits[class_rankings[pos_class_indices]] /
            (1 + class_rankings[pos_class_indices].astype(np.float)))
    return pos_class_indices, precision_at_hits


In [16]:
def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.

    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output

def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear",
                 temperature=1.0):
        super().__init__()

        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.bn_att = nn.BatchNorm1d(out_features)
        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

In [17]:
encoder_params = {
    "tf_efficientnet_b0_ns": {
        "features": 1280,
        "init_op": partial(tf_efficientnet_b0_ns, pretrained=True, drop_path_rate=0.2)
    }
}

In [18]:
def mixup_data(x, y, lam, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''

        
    maxes = torch.max(x, 1) 
    ratio = torch.min(maxes[0]) / maxes[0]
    for i in range(x.shape[0]):
        x[i] = x[i] * ratio[i]

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [19]:
class SEDAudioClassifier(nn.Module):
    
    def __init__(self, mel_params, num_classes=SPECIES_NUM):
        super().__init__()
        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None
        self.interpolate_ratio = 30  # Downsampled ratio
        # load pretrained models, using ResNeSt-50 as an example
        self.encoder = encoder_params['tf_efficientnet_b0_ns']["init_op"]()

        in_features = 1024

        self.fc1 = nn.Linear(1280, in_features, bias=True)
        self.att_block = AttBlock(in_features, num_classes, activation="linear")
        
        
        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=mel_params['n_fft'], hop_length=mel_params['hop_length'], 
            win_length=mel_params['win_length'], window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=SR, n_fft=mel_params['n_fft'], 
            n_mels=mel_params['n_mels'], fmin=mel_params['fmin'], fmax=mel_params['fmax'], ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(mel_params['n_mels'])

        self.init_weight()

    def init_weight(self):
        init_layer(self.fc1)
        init_bn(self.bn0)

    def _add_frequency_encoding(self, x):
        n, d, h, w = x.size()

        vertical = torch.linspace(-1, 1, w, device=x.device).view(1, 1, 1, -1)
        vertical = vertical.repeat(n, 1, h, 1)

        return vertical

    def _add_time_encoding(self, x):
        n, d, h, w = x.size()

        horizontal = torch.linspace(-1, 1, h, device=x.device).view(1, 1, -1, 1)
        horizontal = horizontal.repeat(n, 1, 1, w)

        return horizontal
        
    def forward(self, x, istrain=True):
        
        x = self.spectrogram_extractor(x)
        # batch_size x 1 x time_steps x freq_bins
        x = self.logmel_extractor(x)
        # batch_size x 1 x time_steps x mel_bins

        frames_num = x.shape[2]

        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)

        freq_encode = self._add_frequency_encoding(x)
        time_encode = self._add_time_encoding(x)
        x = torch.cat([x, freq_encode, time_encode], dim=1)
        
        x = self.encoder.forward_features(x)

        # Aggregate in frequency axis
        x = torch.mean(x, dim=3)

        # channel smoothing
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        output_dict = {
            "framewise_output": framewise_output,
            "logit": logit,
            "clipwise_output": clipwise_output
        }

        return output_dict

In [20]:
def get_model(mel_params, num_classes=SPECIES_NUM, is_mean_teacher=False):
    device = 'cuda'
    model = SEDAudioClassifier(mel_params, num_classes=SPECIES_NUM)  
    model = model.to(device)
    # Detach params for Exponential Moving Average Model (aka the Mean Teacher).
    # We'll manually update these params instead of using backprop.
    if is_mean_teacher:
        for param in model.parameters():
            param.detach_()
    return model

In [21]:
# class LitModule(pl.LightningModule):
    
#     def __init__(self, train_len, mel_params, num_classes=SPECIES_NUM):
#         super().__init__()

#         self.trian_criterion = MeanTeacherLoss()
#         self.valid_criterion = bce_criterion
#         self.train_len = train_len
        
#         self.model = get_model(mel_params, num_classes=SPECIES_NUM)
#         self.teacher_model = get_model(mel_params, num_classes=SPECIES_NUM, is_mean_teacher=True)
#         self.ema_decay = 0.995
            
#     def configure_optimizers(self):
#         optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=True)
#         scheduler = {'scheduler': torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=0.001*0.01),
#                                 'name': 'learning_rate',
#                                 'interval':'epoch',
#                                 'frequency': 1}
#         return [optimizer], [scheduler]
    
#     def update_teacher_params(self, alpha, global_step):
#         # Use the true average until the exponential average is more correct
#         alpha = min(1 - 1 / (global_step + 1), alpha)
#         for ema_param, param in zip(self.teacher_model.parameters(), self.model.parameters()):
#             ema_param.data.mul_(alpha).add_(1 - alpha, param.data)
    
    
#     def training_step(self, train_batch, batch_idx):
#         x = train_batch
#         y = x['target']
#         student_input = x['waveform']
#         teacher_input = x['teacher_waveform']
#         classif_weights = x['classification_weights']
#         batch_size = len(y)
        
# #         if np.random.uniform() < 0.2:
# #             alpha=0.5
# #             lam = np.random.beta(alpha, alpha)
# #             student_input, y_a, y_b, lam = mixup_data(student_input, y, alpha=alpha)
# #             y_pred = self.student_model(student_input, istrain=True)
# #             y_pred_t  = self.teacher_model(teacher_input, istrain=True)
# #             loss = mixup_criterion(self.trian_criterion, y_pred['clipwise_output'], y_a, y_b, lam)
# #         else:
#         y_pred = self.model(student_input, istrain=True)
#         y_pred_t  = self.teacher_model(teacher_input, istrain=True)
# #         loss = self.valid_criterion(y_pred['clipwise_output'], y, isTrain=True)
#         loss, class_loss, consistency_loss, consistency_weight = self.trian_criterion(
#                         y_pred['clipwise_output'], y_pred_t['clipwise_output'], y, classif_weights, self.current_epoch)
#         self.update_teacher_params(self.ema_decay, self.global_step%(self.current_epoch+1))

# #             loss = self.trian_criterion(y_pred['clipwise_output'], y)

#         self.log('train_loss', loss,  on_epoch=True, prog_bar=True, logger=True)
#         return loss
    
#     def validation_step(self, val_batch, batch_idx):
#         x, y = val_batch
#         batch_size = len(y)
        
#         y_pred = self.model(x, istrain=False)
#         loss_clip = self.valid_criterion(y_pred['clipwise_output'], y, isTrain=False)
#         loss = loss_clip
#         y_pred_act = y_pred['clipwise_output']
#         lwlap_step, weight_step = lwlap_wrapper(y, y_pred_act)
#         lwlap_step = (lwlap_step * weight_step).sum()
#         self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
#         self.log('lwlap_score', lwlap_step, on_epoch=True, prog_bar=True, logger=True)
#         return loss, lwlap_step

#     def validation_epoch_end(self, validation_step_outputs):
#         validation_step_outputs = np.array(validation_step_outputs)
#         validation_step_losses = validation_step_outputs[:, 0]
#         mean_loss = torch.stack([x for x in validation_step_losses]).mean()
        
#         validation_step_scores = validation_step_outputs[:, 1]
#         mean_score = torch.stack([x for x in validation_step_scores]).mean()

#         print('valid_epoch_loss = ', mean_loss)
#         print('valid_epoch_lwlap = ', mean_score)
#         self.log('valid_epoch_loss', mean_loss, prog_bar=True, logger=True)
#         self.log('valid_epoch_lwlap', mean_score, prog_bar=True, logger=True)
# #         tqdm.write('Dice: \t%.3f' % mean_loss)

#         return mean_loss, mean_score

In [22]:
def pretty_print_metrics(fold, epoch, optimizer, train_loss_metrics, val_loss_metrics):
    print(f"""
    {time.ctime()} \n
    Fold:{fold}, Epoch:{epoch}, LR:{optimizer.param_groups[0]['lr']:.7}, Cons. Weight: {train_loss_metrics['consistency_weight']}\n
    --------------------------------------------------------
    Metric:              Train    |   Val
    --------------------------------------------------------
    Loss:                {train_loss_metrics['loss']:0.4f}   |   {val_loss_metrics['loss']:0.4f}\n
    LWLRAP:              {train_loss_metrics['lwlrap']:0.4f}   |   {val_loss_metrics['lwlrap']:0.4f}\n
    Class Loss:          {train_loss_metrics['class_loss']:0.4f}   | \n
    Consistency Loss:    {train_loss_metrics['consistency_loss']:0.4f}   |\n
    --------------------------------------------------------\n
    """)

In [23]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [24]:
species_id_head_list = []
for l_ in train_gby['species_id']:
    species_id_head_list.append(l_[0])

In [25]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class MetricMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.y_true = []
        self.y_pred = []

    def update(self, y_true, y_pred):
        try:
            self.y_true.extend(y_true.detach().cpu().numpy().tolist())
            self.y_pred.extend(torch.sigmoid(y_pred).cpu().detach().numpy().tolist())
        except:
            print("UPDATE FAILURE")

    def update_list(self, y_true, y_pred):
        self.y_true.extend(y_true)
        self.y_pred.extend(y_pred)

    @property
    def avg(self):
        score_class, weight = lwlrap(np.array(self.y_true), np.array(self.y_pred))
        self.score = (score_class * weight).sum()

        return self.score
    

In [26]:
# Update teacher to be exponential moving average of student params.
def update_teacher_params(student, teacher, alpha, global_step):
    # Use the true average until the exponential average is more correct
    alpha = min(1 - 1 / (global_step + 1), alpha)
    for ema_param, param in zip(teacher.parameters(), student.parameters()):
        ema_param.data.mul_(alpha).add_(1 - alpha, param.data)


def train_one_epoch(student, mean_teacher, loader, 
                    criterion, optimizer, scheduler, epoch):
    global_step = 0
    losses = AverageMeter()
    consistency_loss_avg = AverageMeter()
    class_loss_avg = AverageMeter()
    comp_metric = MetricMeter()
    device = 'cuda'
    ema_decay = 0.995
    
    student.train()
    mean_teacher.train()
#     context = nullcontext()
    
    t = tqdm(loader)
    for i, sample in enumerate(t):
        student_input = sample['waveform'].to(device)
        teacher_input = sample['teacher_waveform'].to(device)
        target = sample['target'].to(device)
        classif_weights = sample['classification_weights'].to(device)
        batch_size = len(target)

        student_pred  = student(student_input)
        teacher_pred  = mean_teacher(teacher_input)

        loss, class_loss, consistency_loss, consistency_weight = criterion(
            student_pred['clipwise_output'], teacher_pred['clipwise_output'], target, classif_weights, epoch)

#         if not is_val:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        update_teacher_params(student, mean_teacher, 
                              ema_decay, global_step)

        scheduler.step()

        comp_metric.update(target, student_pred['clipwise_output'])
        losses.update(loss.item(), batch_size)
        consistency_loss_avg.update(consistency_loss.item(), batch_size)
        class_loss_avg.update(class_loss.item(), batch_size)
        global_step += 1

        t.set_description(f"Epoch:{epoch} - Loss:{losses.avg:0.4f}")
    t.close()
    return {'lwlrap':comp_metric.avg, 
            'loss':losses.avg, 
            'consistency_loss':consistency_loss_avg.avg, 
            'class_loss':class_loss_avg.avg, 
            'consistency_weight':consistency_weight}


def valid_one_epoch(student, mean_teacher, loader, 
                    criterion, optimizer, scheduler, epoch):
    global_step = 0
    losses = AverageMeter()
    consistency_loss_avg = AverageMeter()
    class_loss_avg = AverageMeter()
    comp_metric = MetricMeter()
    device = 'cuda'
    
    student.eval()
    mean_teacher.eval()
    context = torch.no_grad()
#     else:
#     student.train()
#     mean_teacher.train()
#     context = nullcontext()
    
    with context:
        t = tqdm(loader)
        for i, (sample, y) in enumerate(t):
            student_input = sample.to(device)
            y = y.to(device)
            batch_size = len(y)

            student_pred = student(student_input)
#             teacher_pred = mean_teacher(teacher_input)

            loss = criterion(torch.sigmoid(student_pred['clipwise_output']), y, isTrain=False)

#             loss, class_loss, consistency_loss, consistency_weight = criterion(
#                 student_pred['clipwise_output'], teacher_pred['clipwise_output'], target, classif_weights, epoch)

            comp_metric.update(y, student_pred['clipwise_output'])
            losses.update(loss.item(), batch_size)
#             consistency_loss_avg.update(consistency_loss.item(), batch_size)
#             class_loss_avg.update(class_loss.item(), batch_size)
            global_step += 1

            t.set_description(f"Epoch:{epoch} - Loss:{losses.avg:0.4f}")
        t.close()
    return {'lwlrap':comp_metric.avg, 
            'loss':losses.avg}

In [27]:
def train(mel_params, train_loader, val_loader, fold):
#     train_df = df[df.fold != fold]
#     val_df = df[df.fold == fold]
#     train_loader = get_data_loader(train_df, config.train_batch_size, is_val=False)
#     val_loader = get_data_loader(val_df, config.valid_batch_size, is_val=True)

    student_model = get_model(mel_params, num_classes=SPECIES_NUM)
    teacher_model = get_model(mel_params, num_classes=SPECIES_NUM, is_mean_teacher=True)

    optimizer = torch.optim.AdamW(student_model.parameters(), lr=0.001)
    num_train_steps = int(len(train_loader) * EPOCH)
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=0.001*0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_train_steps)
    train_criterion = MeanTeacherLoss()
    valid_criterion = bce_criterion

    best_val_metric = -np.inf
    val_metrics = []
    train_metrics = []
    for epoch in range(0, EPOCH):
        train_loss_metrics = train_one_epoch(
            student_model, teacher_model, train_loader, 
            train_criterion, optimizer, scheduler, epoch)
        val_loss_metrics = valid_one_epoch(
            student_model, teacher_model, val_loader, 
            valid_criterion, optimizer, scheduler, epoch)

        train_metrics.append(train_loss_metrics)
        val_metrics.append(val_loss_metrics)
        pretty_print_metrics(fold, epoch, optimizer, 
                             train_loss_metrics, val_loss_metrics)
        
        if val_loss_metrics['lwlrap'] > best_val_metric:
            print(f"    LWLRAP Improved from {best_val_metric} --> {val_loss_metrics['lwlrap']}\n")
            torch.save(teacher_model.state_dict(), 
                       os.path.join(config.save_path, f'fold-{fold}.bin'))
            best_val_metric = val_loss_metrics['lwlrap']
    return student_model, teacher_model
    

In [28]:
for fold_id, (train_index, val_index) in enumerate(skf.split(train_gby, species_id_head_list)):
    # Picking only first fold to train/val on
    # This means loss of 20% training data
    # To avoid this, you can train 5 different models on 5 folds and average predictions
    train_data = train_df[~train_df['name'].isin(train_gby.iloc[val_index]['name'])]
    valid_data = train_df[train_df['name'].isin(train_gby.iloc[val_index]['name'])]    
    
    train_data.reset_index(inplace=True, drop=True)
    valid_data.reset_index(drop=True, inplace=True)
    
    train_dataset = SpectrogramTrainDataset(train_data, config["dataset"]["params"])
    valid_dataset = SpectrogramValidDataset(valid_data, config["dataset"]["params"])
    
    train_loader = data.DataLoader(train_dataset, **config["loader"]["train"])
    valid_loader = data.DataLoader(valid_dataset, **config["loader"]["valid"])
    
    train_len = len(train_loader)
    
    student_model, teacher_model = train(config['dataset']['params']['melspectrogram_parameters'], train_loader, valid_loader, fold_id)
        
#     model = LitModule(train_len, config['dataset']['params']['melspectrogram_parameters'])
    
#     trainer = pl.Trainer(
#         max_epochs=EPOCH,
#         default_root_dir=OUTPUT_DIR,
#         gpus=1,
# #         callbacks=[TrainCallback],
#         deterministic=True,
#         benchmark=True
#     )
#     trainer.fit(model, train_loader, valid_loader)
    
    break
    
    torch.save(model.state_dict(), OUTPUT_DIR + 'model' + str(fold_id))

/usr/local/lib/python3.6/dist-packages/librosa/filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
  0%|          | 0/324 [00:00<?, ?it/s]/home/user/.local/lib/python3.6/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  
  0%|          | 0/324 [00:00<?, ?it/s]


    Sun Feb  7 19:57:26 2021 

    Fold:0, Epoch:0, LR:0.0009990134, Cons. Weight: 0.6737946999085467

    --------------------------------------------------------
    Metric:              Train    |   Val
    --------------------------------------------------------
    Loss:                7.3587   |   0.1951

    LWLRAP:              0.2094   |   0.3279

    Class Loss:          7.3529   | 

    Consistency Loss:    0.0059   |

    --------------------------------------------------------

    
    LWLRAP Improved from -inf --> 0.3279046927423459



  0%|          | 0/324 [00:00<?, ?it/s]


    Sun Feb  7 20:01:21 2021 

    Fold:0, Epoch:1, LR:0.0009960574, Cons. Weight: 0.6805630463993154

    --------------------------------------------------------
    Metric:              Train    |   Val
    --------------------------------------------------------
    Loss:                6.4765   |   0.1857

    LWLRAP:              0.2987   |   0.3520

    Class Loss:          6.4721   | 

    Consistency Loss:    0.0044   |

    --------------------------------------------------------

    
    LWLRAP Improved from 0.3279046927423459 --> 0.35201177489208735



  0%|          | 0/324 [00:00<?, ?it/s]

...

  0%|          | 0/324 [00:00<?, ?it/s]


    Sun Feb  7 23:06:10 2021 

    Fold:0, Epoch:48, LR:9.866358e-07, Cons. Weight: 1.0764301892641972

    --------------------------------------------------------
    Metric:              Train    |   Val
    --------------------------------------------------------
    Loss:                0.9085   |   0.0571

    LWLRAP:              0.9325   |   0.8906

    Class Loss:          0.9030   | 

    Consistency Loss:    0.0055   |

    --------------------------------------------------------

    


Epoch:49 - Loss:0.0574: 100%|██████████| 121/121 [00:02<00:00, 44.42it/s]



    Sun Feb  7 23:10:06 2021 

    Fold:0, Epoch:49, LR:0.0, Cons. Weight: 1.086721304855215

    --------------------------------------------------------
    Metric:              Train    |   Val
    --------------------------------------------------------
    Loss:                0.8925   |   0.0574

    LWLRAP:              0.9257   |   0.8784

    Class Loss:          0.8873   | 

    Consistency Loss:    0.0052   |

    --------------------------------------------------------

    


NameError: name 'model' is not defined

In [25]:
torch.save(model.state_dict(), OUTPUT_DIR + 'model')

In [ ]:
base_model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
layers = list(base_model.children())[:-1]

in_features = base_model.fc.in_features
in_features

In [20]:
a

NameError: name 'a' is not defined